In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import numpy.linalg as la
from tqdm.notebook import tqdm

In [45]:
input_iris_path = "../data/raw/iris_2017/CONTOURS-IRIS.shp"
input_reference_path = "../data/raw/flow/referentiel-comptages-routiers.shp"
input_simulation_path = "../data/ile_de_france_network.shp"

output_path = "../data/network_matching.csv"
output_comparison_reference = "../data/comarison_reference.gpkg"
output_comparison_simulation = "../data/comarison_simulation.gpkg"

if "snakemake" in locals():
    input_iris_path = snakemake.input["iris"]
    input_reference_path = snakemake.input["reference"]
    input_simulation_path = snakemake.input["simulation"]

    output_path = snakemake.output["matching"]
    
    output_comparison_reference = snakemake.output["comparison_reference"] if "comparison_reference" in snakemake.output else None
    output_comparison_simulation = snakemake.output["comparison_simulation"] if "comparison_simulation" in snakemake.output else None

In [3]:
# Obtain shape for Paris to filter out network links
df_spatial = gpd.read_file(input_iris_path)
df_spatial = df_spatial[df_spatial["INSEE_COM"].str.startswith("75")]
df_spatial = df_spatial.dissolve()[["geometry"]]

In [4]:
# Load reference link data
df_reference = gpd.read_file(input_reference_path)
df_reference = df_reference.rename(columns = {
    "iu_ac": "reference_id"
})[["reference_id", "geometry"]]
df_reference = df_reference.to_crs("epsg:2154")

In [5]:
# Load simulation link data
df_simulation = gpd.read_file(input_simulation_path)
df_simulation = df_simulation.rename(columns = {
    "link": "simulation_id"
})[["simulation_id", "geometry", "osm"]]

In [6]:
# Filter links for Paris
df_simulation = gpd.sjoin(df_simulation, df_spatial, op = "within")

In [7]:
# Filter for higher order roads
df_simulation = df_simulation[
    df_simulation["osm"].str.contains("motorway") |
    df_simulation["osm"].str.contains("trunk") |
    df_simulation["osm"].str.contains("primary") |
    df_simulation["osm"].str.contains("secondary") |
    df_simulation["osm"].str.contains("tertiary")
]

df_simulation = df_simulation[["simulation_id", "geometry"]]

In [8]:
# Calculate centroids
reference_centroids = np.vstack([df_reference["geometry"].centroid.x, df_reference["geometry"].centroid.y]).T
simulation_centroids = np.vstack([df_simulation["geometry"].centroid.x, df_simulation["geometry"].centroid.y]).T

In [9]:
# Calculate orientation

def angle(geometry):
    coordinates = np.array(geometry.xy).T
    return np.arctan2(coordinates[-1, 1] - coordinates[0, 1], coordinates[-1, 0] - coordinates[0, 0])
    
reference_angles = df_reference["geometry"].apply(angle).values
simulation_angles = df_simulation["geometry"].apply(angle).values

In [10]:
# Calculate distances

centroid_distances = np.zeros((len(reference_centroids), len(simulation_centroids)))
angle_distances = np.zeros((len(reference_centroids), len(simulation_centroids)))

for k in tqdm(range(len(reference_centroids))):
    centroid_distances[k,:] = la.norm(reference_centroids[k] - simulation_centroids, axis = 1)
    angle_distances[k,:] = np.abs(reference_angles[k] - simulation_angles)
    
angle_distances[angle_distances < 0.0] += 2.0 * np.pi

  0%|          | 0/3739 [00:00<?, ?it/s]

In [30]:
# Scoring
alpha = 0.1
scores = centroid_distances + alpha * angle_distances

In [31]:
# Filtering

maximum_distance = 50
maximum_angle = 15 * np.pi / 180.0

scores[centroid_distances > maximum_distance] = np.inf
scores[angle_distances > maximum_angle] = np.inf

In [32]:
# Matching process
matchings = []

while np.count_nonzero(np.isfinite(scores)) > 0:
    index = np.unravel_index(np.argmin(scores), scores.shape)

    scores[index[0], :] = np.inf
    scores[:, index[1]] = np.inf

    matchings.append(index)
    
matchings = np.array(matchings)

In [43]:
df_matching = pd.DataFrame({
    "reference_id": df_reference.iloc[matchings[:,0]]["reference_id"].values,
    "simulation_id": df_simulation.iloc[matchings[:,1]]["simulation_id"].values
})

df_matching.to_csv(output_path, sep = ";", index = False)

In [44]:
# Write comparison

if not output_comparison_reference is None:
    df_comparison = df_reference.copy()
    df_comparison = pd.merge(df_comparison, df_matching)
    df_comparison.to_file(output_comparison_reference, driver = "GPKG")
    
if not output_comparison_simulation is None:
    df_comparison = df_simulation.copy()
    df_comparison = pd.merge(df_comparison, df_matching)
    df_comparison.to_file(output_comparison_simulation, driver = "GPKG")